In [1]:
from datetime import datetime
from dateutil import tz
import json
import sys
import requests
import pprint

In [2]:
def get_schedule(year, team):
    # print('Get schedule for ' + str(team))
    url = 'https://statsapi.web.nhl.com/api/v1/schedule?startDate=' + str(year) + '-10-01&endDate=' + str(year + 1) + '-05-29&expand=schedule.teams,schedule.linescore,schedule.broadcasts,schedule.ticket,schedule.game.content.media.epg&leaderCategories=&site=en_nhlCA&teamId=' + str(team)
    team_schedule = requests.get(url)
    return team_schedule.json()

def get_playoff_schedule(year, team):
    url = 'https://statsapi.web.nhl.com/api/v1/schedule?startDate=' + str(year + 1) + '-04-01&endDate=' + str(year + 1) + '-10-15&expand=schedule.teams,schedule.linescore,&site=en_nhlCA&teamId=' + str(team)
    # print(url)
    team_schedule = requests.get(url)
    return team_schedule.json()

def get_live_result(link):
    url = 'https://statsapi.web.nhl.com' + link
    live = requests.get(url).json()
    return live

def get_team(id):
    url = 'https://statsapi.web.nhl.com/api/v1/teams/' + str(id)
    team = requests.get(url).json()
    return team['teams'][0]

def get_teams(year):
    ystr = str(year) + str(year + 1)
    url = 'https://statsapi.web.nhl.com/api/v1/standings?season=' + ystr
    standings = requests.get(url).json()
    teams = {}
    for record in standings["records"]:
        for team in record['teamRecords']:
            info = get_team(team['team']['id'])
            team_record = {'info': info, 'standings': team, 'schedule': []}
            teams[team['team']['id']] = team_record
    return teams

def get_round_robin_teams(year):
    ystr = str(year) + str(year + 1)
    url = 'https://statsapi.web.nhl.com/api/v1/standings/byConference?season=' + ystr + '&rr=true'
    standings = requests.get(url).json()
    teams = {}
    for record in standings["records"]:
        for team in record['teamRecords']:
            info = get_team(team['team']['id'])
            team_record = {'info': info, 'standings': team, 'schedule': []}
            teams[team['team']['id']] = team_record
    return teams

In [3]:
class Updater(object):

    def __init__(self, server, year):
        self._server = server
        self._year = year
        self._current_round = 0
        self._teams = {}
        self._matchups = {}
        self._standings = {}
        self._round_robin_standing = {}
        self._qualif_east = []
        self._qualif_west = []
        self.load()

    def run(self):
        if self._current_round == 0:
            self._teams = self.get_teams()
            self._matchups = self.create_matchups_tree()

            if len(list(self._teams.values())) > 0:
                # Determine the qualifications matchup
                round_robin_team = self.get_round_robin_teams()
                self.update_round_robin_teams(round_robin_team)
                self._standings = self.get_standings(list(self._teams.values()))
                self._round_robin_standing = self.get_round_robin_standings(list(round_robin_team.values()))
                self._qualif_east, self._qualif_west = self.create_qualification_matchups(self._standings)

            if self.is_round_finished(0):
                print('Playoff starting')
                self.create_matchups(self.get_round_winners_standings(0), 0)
                self._current_round = 1
                self.store()
            else:
                print("Storing prelim results")
                self._current_round = 0
                print("Qualif East")
                for matchup in self._qualif_east:
                    if self.is_matchup_finished(matchup, 3):
                        winner = self.get_matchup_winner(matchup, 3)
                        if winner == matchup["home"]:
                            print('{2} {0} VS {3} {1} W:{0}'.format(self._teams[matchup['home']]['info']['abbreviation'], self._teams[matchup['away']]['info']['abbreviation'], matchup['result']['home_win'], matchup['result']['away_win']))
                        else:
                            print('{2} {0} VS {3} {1} W:{1}'.format(self._teams[matchup['home']]['info']['abbreviation'], self._teams[matchup['away']]['info']['abbreviation'], matchup['result']['home_win'], matchup['result']['away_win']))
                    else:
                        print('{2} {0} VS {3} {1}'.format(self._teams[matchup['home']]['info']['abbreviation'], self._teams[matchup['away']]['info']['abbreviation'], matchup['result']['home_win'], matchup['result']['away_win']))
                print("Qualif West")
                for matchup in self._qualif_west:
                    if self.is_matchup_finished(matchup, 3):
                        winner = self.get_matchup_winner(matchup, 3)
                        if winner == matchup["home"]:
                            print('{2} {0} VS {3} {1} W:{0}'.format(self._teams[matchup['home']]['info']['abbreviation'], self._teams[matchup['away']]['info']['abbreviation'], matchup['result']['home_win'], matchup['result']['away_win']))
                        else:
                            print('{2} {0} VS {3} {1} W:{1}'.format(self._teams[matchup['home']]['info']['abbreviation'], self._teams[matchup['away']]['info']['abbreviation'], matchup['result']['home_win'], matchup['result']['away_win']))
                    else:
                        print('{2} {0} VS {3} {1}'.format(self._teams[matchup['home']]['info']['abbreviation'], self._teams[matchup['away']]['info']['abbreviation'], matchup['result']['home_win'], matchup['result']['away_win']))

                print("Round robin East")
                for team in self._round_robin_standing['Eastern']['teams']:
                    print("{0} - {5} {1} {2} {3} {4}".format(team['info']['abbreviation'], team['standings']['leagueRecord']["wins"], team['standings']['leagueRecord']["losses"], team['standings']['leagueRecord']["ot"], team['standings']['points'], team['standings']['gamesPlayed']))
                print("Round robin West")
                for team in self._round_robin_standing['Western']['teams']:
                    print("{0} - {5} {1} {2} {3} {4}".format(team['info']['abbreviation'], team['standings']['leagueRecord']["wins"], team['standings']['leagueRecord']["losses"], team['standings']['leagueRecord']["ot"], team['standings']['points'], team['standings']['gamesPlayed']))
                self.store()
        elif self._current_round == 1:
            self.update_matchups()
            c1 = self._matchups['c1']
            c2 = self._matchups['c2']
            p1 = self._matchups['p1']
            p2 = self._matchups['p2']
            c = self._matchups['c']
            p = self._matchups['p']
            a1 = self._matchups['a1']
            a2 = self._matchups['a2']
            m1 = self._matchups['m1']
            m2 = self._matchups['m2']
            a = self._matchups['a']
            m = self._matchups['m']
            r1wnext = [(c, True), (p, True), (p,False), (c, False)]
            r1enext = [(a, True), (m, True), (a,False), (m, False)]
            r1w = [c1, c2, p1, p2]
            r1e = [a1, a2, m1, m2]
            for m in r1w:
                if self.is_matchup_finished(m):
                    winner = self.get_matchup_winner(m)
                    if winner == m['home']:
                        print("Home winner", m["id"], winner)
                        self.update_round_home(winner, r1wnext)
                    else:
                        print("Away winner", m["id"])
                        self.update_round_away(winner, r1wnext)
            
            for m in r1e:
                if self.is_matchup_finished(m):
                    winner = self.get_matchup_winner(m)
                    if winner == m['home']:
                        print("Home winner", m["id"], winner)
                        self.update_round_home(winner, r1enext)
                    else:
                        print("Away winner", m["id"])
                        self.update_round_away(winner, r1enext)
            
        else:
            self.update_matchups()
            if self.is_round_finished(self._current_round):
                self.create_matchups(self.get_round_winners_standings(self._current_round), self._current_round)
                self._current_round += 1
            self.store()

    def is_round_finished(self, round):
        if round == 0:
            if self.is_qualification_finished(self._qualif_east, self._qualif_west) and self.is_round_robin_finished():
                return True
        else:
            matchups = self.get_matchup_round(round)
            for matchup in matchups:
                if not self.is_matchup_finished(matchup):
                    return False
            return True
        return False

    def update_round_home(self, winner, rnext):
        for r in rnext:
            if r[1]:
                if r[0]["home"] == 0:
                    print("Set Matchup home", r[0]['id'], r[0]['home'], r[0]['away'], winner)
                    self.update_matchup(r[0], winner)
                    return
            else:
                if r[0]["away"] == 0:
                    print("Set Matchup away", r[0]['id'], r[0]['home'], r[0]['away'], winner)
                    self.update_matchup(r[0], winner)
                    return

    def update_round_away(self, winner, rnext):
        rnr = rnext.copy()
        rnr.reverse()
        for r in rnr:
            if r[1]:
                if r[0]["home"] == 0:
                    print("Set Matchup home", r[0]['id'], r[0]['home'], r[0]['away'], winner)
                    self.update_matchup(r[0], winner)
                    return
            else:
                if r[0]["away"] == 0:
                    print("Set Matchup away", r[0]['id'], r[0]['home'], r[0]['away'], winner)
                    self.update_matchup(r[0], winner)
                    return
            
    def get_matchup_round(self, round):
        matchups = []
        for matchup in self._matchups.values():
            if matchup['round'] == round:
                matchups.append(matchup)
        return matchups

    def update_round_robin_teams(self, teams):
        for team in teams.values():
            self._teams[team['info']['id']] = team

    def create_qualification_matchups(self, standings):
        i = 1
        qe1 = self.create_matchup('qe1', 0, None)
        self.update_matchup(qe1, standings['Eastern']['teams'][4]['info']['id'], standings['Eastern']['teams'][11]['info']['id'])
        qe2 = self.create_matchup('qe2', 0, None)
        self.update_matchup(qe2, standings['Eastern']['teams'][5]['info']['id'], standings['Eastern']['teams'][10]['info']['id'])
        qe3 = self.create_matchup('qe3', 0, None)
        self.update_matchup(qe3, standings['Eastern']['teams'][6]['info']['id'], standings['Eastern']['teams'][9]['info']['id'])
        qe4 = self.create_matchup('qe4', 0, None)
        self.update_matchup(qe4, standings['Eastern']['teams'][7]['info']['id'], standings['Eastern']['teams'][8]['info']['id'])
        qw1 = self.create_matchup('qw1', 0, None)
        self.update_matchup(qw1, standings['Western']['teams'][4]['info']['id'], standings['Western']['teams'][11]['info']['id'])
        qw2 = self.create_matchup('qw2', 0, None)
        self.update_matchup(qw2, standings['Western']['teams'][5]['info']['id'], standings['Western']['teams'][10]['info']['id'])
        qw3 = self.create_matchup('qw3', 0, None)
        self.update_matchup(qw3, standings['Western']['teams'][6]['info']['id'], standings['Western']['teams'][9]['info']['id'])
        qw4 = self.create_matchup('qw4', 0, None)
        self.update_matchup(qw4, standings['Western']['teams'][7]['info']['id'], standings['Western']['teams'][8]['info']['id'])
        return [qe1, qe2, qe3, qe4], [qw1, qw2, qw3, qw4]

    def is_round_robin_finished(self):
        for team in self._round_robin_standing['Eastern']['teams']:
            if team['standings']['gamesPlayed'] != 3:
                return False
        for team in self._round_robin_standing['Western']['teams']:
            if team['standings']['gamesPlayed'] != 3:
                return False
        return True

    def is_qualification_finished(self, east, west):
        for matchup in east:
            if not self.is_matchup_finished(matchup, 3):
                return False
        for matchup in west:
            if not self.is_matchup_finished(matchup, 3):
                return False
        return True

    def get_qualification_winners(self, east, west):
        east_winners = []
        west_winners = []
        for matchup in east:
            if self.is_matchup_finished(matchup, 3):
                east_winners.append(self._teams[self.get_matchup_winner(matchup, 3)])
        for matchup in west:
            if self.is_matchup_finished(matchup, 3):
                west_winners.append(self._teams[self.get_matchup_winner(matchup, 3)])
        return east_winners, west_winners

    def is_season_finished(self):
        for team in list(self._teams.values()):
            remaining = 82 - team['standings']['gamesPlayed']
            if remaining > 0:
                return False
        return True

    def get_round_standings(self, round):
        teams = []
        matchups = self.get_matchup_round(round)
        for matchup in matchups:
            teams.append(self._teams[matchup['home']])
            teams.append(self._teams[matchup['away']])
        standings = {'Eastern': {'Scotia North': [], 'Metropolitan': [], 'teams': []},
                     'Western': {'Central': [], 'Pacific': [], 'teams': []},
                     'teams': []}
        league = sorted(teams, key=lambda k: int(k['standings']['conferenceRank']))
        for team in league:
            standings['teams'].append(team)
            standings[team['info']['conference']['name']]['teams'].append(team)
            standings[team['info']['conference']['name']][team['info']['division']['name']].append(team)
        standings['teams'] = sorted(standings['teams'], key=lambda k: int(k['standings']['leagueRank']))

        standings['Eastern']['teams'] = sorted(standings['Eastern']['teams'], key=lambda k: int(k['standings']['conferenceRank']))
        standings['Western']['teams'] = sorted(standings['Western']['teams'], key=lambda k: int(k['standings']['conferenceRank']))

        standings['Eastern']['Scotia North'] = sorted(standings['Eastern']['Scotia North'], key=lambda k: int(k['standings']['conferenceRank']))
        standings['Eastern']['Metropolitan'] = sorted(standings['Eastern']['Metropolitan'], key=lambda k: int(k['standings']['conferenceRank']))
        standings['Western']['Central'] = sorted(standings['Western']['Central'], key=lambda k: int(k['standings']['conferenceRank']))
        standings['Western']['Pacific'] = sorted(standings['Western']['Pacific'], key=lambda k: int(k['standings']['conferenceRank']))
        return standings

    def get_round_winners_standings(self, round):
        if round == 0:
            teams = []
            east_winners, west_winners = self.get_qualification_winners(self._qualif_east, self._qualif_west)
            for team in east_winners:
                print(team['info']['name'])
                teams.append(team)
            for team in west_winners:
                print(team['info']['name'])
                teams.append(team)
            for team in self._round_robin_standing['teams']:
                print(team['info']['name'])
                teams.append(team)
        else:
            teams = []
            matchups = self.get_matchup_round(round)
            for matchup in matchups:
                winner = self.get_matchup_winner(matchup)
                teams.append(self._teams[winner])
        standings = {'Eastern': {'Scotia North': [], 'Metropolitan': [], 'teams': []},
                     'Western': {'Central': [], 'Pacific': [], 'teams': []},
                     'teams': []}
        league = sorted(teams, key=lambda k: int(k['standings']['conferenceRank']))
        for team in league:
            standings['teams'].append(team)
            standings[team['info']['conference']['name']]['teams'].append(team)
            standings[team['info']['conference']['name']][team['info']['division']['name']].append(team)
        standings['teams'] = sorted(standings['teams'], key=lambda k: int(k['standings']['leagueRank']))

        standings['Eastern']['teams'] = sorted(standings['Eastern']['teams'], key=lambda k: int(k['standings']['conferenceRank']))
        standings['Western']['teams'] = sorted(standings['Western']['teams'], key=lambda k: int(k['standings']['conferenceRank']))

        standings['Eastern']['Scotia North'] = sorted(standings['Eastern']['Scotia North'], key=lambda k: int(k['standings']['conferenceRank']))
        standings['Eastern']['Metropolitan'] = sorted(standings['Eastern']['Metropolitan'], key=lambda k: int(k['standings']['conferenceRank']))
        standings['Western']['Central'] = sorted(standings['Western']['Central'], key=lambda k: int(k['standings']['conferenceRank']))
        standings['Western']['Pacific'] = sorted(standings['Western']['Pacific'], key=lambda k: int(k['standings']['conferenceRank']))
        print(len(standings['Eastern']['teams']))
        return standings

    def create_matchups(self, standings, round):
        if round == 0:
            self.create_matchups_round0(standings)
        elif round == 1:
            self.create_matchups_round1(standings)
        elif round == 2:
            self.create_matchups_round2(standings)
        elif round == 3:
            self.create_matchups_round3(standings)

    def create_matchups_round0(self, standings):
        a1 = self._matchups['a1']
        a2 = self._matchups['a2']
        m1 = self._matchups['m1']
        m2 = self._matchups['m2']
        self.update_matchup(a1, standings['Eastern']['teams'][0]['info']['id'], standings['Eastern']['teams'][7]['info']['id'])
        self.update_matchup(a2, standings['Eastern']['teams'][1]['info']['id'], standings['Eastern']['teams'][6]['info']['id'])
        self.update_matchup(m1, standings['Eastern']['teams'][2]['info']['id'], standings['Eastern']['teams'][5]['info']['id'])
        self.update_matchup(m2, standings['Eastern']['teams'][3]['info']['id'], standings['Eastern']['teams'][4]['info']['id'])
        c1 = self._matchups['c1']
        c2 = self._matchups['c2']
        p1 = self._matchups['p1']
        p2 = self._matchups['p2']
        self.update_matchup(c1, standings['Western']['teams'][0]['info']['id'], standings['Western']['teams'][7]['info']['id'])
        self.update_matchup(c2, standings['Western']['teams'][1]['info']['id'], standings['Western']['teams'][6]['info']['id'])
        self.update_matchup(p1, standings['Western']['teams'][2]['info']['id'], standings['Western']['teams'][5]['info']['id'])
        self.update_matchup(p2, standings['Western']['teams'][3]['info']['id'], standings['Western']['teams'][4]['info']['id'])

    def create_matchups_round1(self, standings):
        a = self._matchups['a']
        m = self._matchups['m']
        self.update_matchup(a, standings['Eastern']['teams'][0]['info']['id'], standings['Eastern']['teams'][3]['info']['id'])
        self.update_matchup(m, standings['Eastern']['teams'][1]['info']['id'], standings['Eastern']['teams'][2]['info']['id'])
        c = self._matchups['c']
        p = self._matchups['p']
        self.update_matchup(c, standings['Western']['teams'][0]['info']['id'], standings['Western']['teams'][3]['info']['id'])
        self.update_matchup(p, standings['Western']['teams'][1]['info']['id'], standings['Western']['teams'][2]['info']['id'])

    def create_matchups_round2(self, standings):
        e = self._matchups['e']
        self.update_matchup(e, standings['Eastern']['teams'][0]['info']['id'], standings['Eastern']['teams'][1]['info']['id'])
        w = self._matchups['w']
        self.update_matchup(w, standings['Western']['teams'][0]['info']['id'], standings['Western']['teams'][1]['info']['id'])

    def create_matchups_round3(self, standings):
        sc = self._matchups['sc']
        self.update_matchup(sc, standings['Eastern']['teams'][0]['info']['id'], standings['Western']['teams'][0]['info']['id'])

    def set_matchup_childs(self, matchup, right, left):
        matchup['left'] = left
        matchup['right'] = right

    def create_matchup(self, id, round, next):
        matchup = {'id': id, 'home': 0, 'away': 0, 'round': round, 'start': '', 'result': {}, 'schedule': [], 'season': {}, 'next': next}
        matchup['left'] = None
        matchup['right'] = None
        matchup['result'] = {'home_win': 0, 'away_win': 0}
        return matchup

    def create_matchups_tree(self):
        matchups = {}
        sc = self.create_matchup('sc', 4, None)
        matchups[sc['id']] = sc

        e = self.create_matchup('e', 3, sc)
        w = self.create_matchup('w', 3, sc)
        matchups[e['id']] = e
        matchups[w['id']] = w

        a = self.create_matchup('n', 2, e)
        m = self.create_matchup('m', 2, e)
        c = self.create_matchup('e', 2, w)
        p = self.create_matchup('w', 2, w)
        matchups[a['id']] = a
        matchups[m['id']] = m
        matchups[c['id']] = c
        matchups[p['id']] = p

        a1 = self.create_matchup('a1', 1, a)
        a2 = self.create_matchup('a2', 1, a)
        m1 = self.create_matchup('m1', 1, m)
        m2 = self.create_matchup('m2', 1, m)
        c1 = self.create_matchup('c1', 1, c)
        c2 = self.create_matchup('c2', 1, c)
        p1 = self.create_matchup('p1', 1, p)
        p2 = self.create_matchup('p2', 1, p)
        matchups[a1['id']] = a1
        matchups[a2['id']] = a2
        matchups[m1['id']] = m1
        matchups[m2['id']] = m2
        matchups[c1['id']] = c1
        matchups[c2['id']] = c2
        matchups[p1['id']] = p1
        matchups[p2['id']] = p2

        # build tree
        self.set_matchup_childs(sc, e, w)

        self.set_matchup_childs(w, p, c)
        self.set_matchup_childs(e, m, a)

        self.set_matchup_childs(c, c2, c1)
        self.set_matchup_childs(p, p2, p1)
        self.set_matchup_childs(a, a2, a1)
        self.set_matchup_childs(m, m2, m1)

        return matchups

    def get_live_result(self, link):
        url = 'https://statsapi.web.nhl.com' + link
        live = requests.get(url).json()
        return live

    def get_team(self, id):
        url = 'https://statsapi.web.nhl.com/api/v1/teams/' + str(id)
        team = requests.get(url).json()
        return team['teams'][0]

    def get_teams(self):
        ystr = str(self._year) + str(self._year + 1)
        url = 'https://statsapi.web.nhl.com/api/v1/standings?season=' + ystr
        standings = requests.get(url).json()
        teams = {}
        for record in standings["records"]:
            for team in record['teamRecords']:
                info = self.get_team(team['team']['id'])
                team_record = {'info': info, 'standings': team, 'schedule': []}
                teams[team['team']['id']] = team_record
        return teams

    def get_round_robin_teams(self):
        ystr = str(self._year) + str(self._year + 1)
        url = 'https://statsapi.web.nhl.com/api/v1/standings/byConference?season=' + ystr + '&rr=true'
        standings = requests.get(url).json()
        teams = {}
        for record in standings["records"]:
            for team in record['teamRecords']:
                info = self.get_team(team['team']['id'])
                team_record = {'info': info, 'standings': team, 'schedule': []}
                teams[team['team']['id']] = team_record
        return teams

    def get_round_robin_standings(self, teams):
        standings = {'Eastern': {'Scotia North': [], 'Metropolitan': [], 'teams': []},
                     'Western': {'Central': [], 'Pacific': [], 'teams': []},
                     'teams': []}

        league = sorted(teams, key=lambda k: int(k['standings']['conferenceRank']))
        for team in league:
            standings['teams'].append(team)
            standings[team['info']['conference']['name']]['teams'].append(team)
            standings[team['info']['conference']['name']][team['info']['division']['name']].append(team)
        standings['teams'] = sorted(standings['teams'], key=lambda k: int(k['standings']['leagueRank']))

        standings['Eastern']['teams'] = sorted(standings['Eastern']['teams'], key=lambda k: int(k['standings']['conferenceRank']))
        standings['Western']['teams'] = sorted(standings['Western']['teams'], key=lambda k: int(k['standings']['conferenceRank']))

        standings['Eastern']['Scotia North'] = sorted(standings['Eastern']['Scotia North'], key=lambda k: int(k['standings']['conferenceRank']))
        standings['Eastern']['Metropolitan'] = sorted(standings['Eastern']['Metropolitan'], key=lambda k: int(k['standings']['conferenceRank']))
        standings['Western']['Central'] = sorted(standings['Western']['Central'], key=lambda k: int(k['standings']['conferenceRank']))
        standings['Western']['Pacific'] = sorted(standings['Western']['Pacific'], key=lambda k: int(k['standings']['conferenceRank']))

        return standings

    def get_standings(self, teams):
        standings = {'Eastern': {'Scotia North': [], 'Metropolitan': [], 'teams': []},
                     'Western': {'Central': [], 'Pacific': [], 'teams': []},
                     'teams': []}

        league = sorted(teams, key=lambda k: int(k['standings']['conferenceRank']))
        for team in league:
            standings['teams'].append(team)
            standings[team['info']['conference']['name']]['teams'].append(team)
            standings[team['info']['conference']['name']][team['info']['division']['name']].append(team)
        standings['teams'] = sorted(standings['teams'], key=lambda k: int(k['standings']['leagueRank']))

        standings['Eastern']['teams'] = sorted(standings['Eastern']['teams'], key=lambda k: int(k['standings']['conferenceRank']))
        standings['Western']['teams'] = sorted(standings['Western']['teams'], key=lambda k: int(k['standings']['conferenceRank']))

        standings['Eastern']['Scotia North'] = sorted(standings['Eastern']['Scotia North'], key=lambda k: int(k['standings']['conferenceRank']))
        standings['Eastern']['Metropolitan'] = sorted(standings['Eastern']['Metropolitan'], key=lambda k: int(k['standings']['conferenceRank']))
        standings['Western']['Central'] = sorted(standings['Western']['Central'], key=lambda k: int(k['standings']['conferenceRank']))
        standings['Western']['Pacific'] = sorted(standings['Western']['Pacific'], key=lambda k: int(k['standings']['conferenceRank']))

        return standings

    def parse_time(self, timestamp):
        from_zone = tz.gettz('UTC')
        to_zone = tz.gettz('America/New_York')
        utc = datetime.strptime(timestamp, '%Y-%m-%dT%H:%M:%SZ')
        utc = utc.replace(tzinfo=from_zone)
        return utc.astimezone(to_zone)

    def get_matchup_schedule(self, matchup, schedules=None):
        home_id = matchup['home']
        away_id = matchup['away']
        result = []
        s = schedules
        if schedules is None:
            s = self.get_playoff_schedule(int(home_id))
        if 'dates' in s:
            for date in s['dates']:
                game = date['games'][0]
                game_home_id = game['teams']['home']['team']['id']
                game_away_id = game['teams']['away']['team']['id']
                if game['gameType'] == 'P':
                    if game_home_id == away_id or game_away_id == away_id:
                        result.append(game)
        else:
            print('No date in get matchup')
        result = sorted(result, key=lambda k: self.parse_time(k['gameDate']))
        return result

    def get_matchup_start(self, matchup):
        if len(matchup['schedule']) == 0:
            return ''
        return matchup['schedule'][0]['gameDate']

    def get_matchup_season_result(self, home, away):
        result = {'home_win': 0, 'away_win': 0, 'matchs': []}
        schedule = self._teams[home]['schedule']
        if len(schedule) == 0:
            schedule = self.get_schedule(home)
            # self._teams[home]['schedule'] = schedule

        if 'dates' in schedule:
            for date in schedule['dates']:
                game = date['games'][0]
                game_home_id = game['teams']['home']['team']['id']
                game_away_id = game['teams']['away']['team']['id']
                if game_home_id == away:
                    print(game['gameDate'], game['teams']['away']['score'], game['teams']['home']['score'])
                    if int(game['teams']['home']['score']) > int(game['teams']['away']['score']):
                        result['away_win'] = result['away_win'] + 1
                    elif int(game['teams']['home']['score']) < int(game['teams']['away']['score']):
                        result['home_win'] = result['home_win'] + 1
                    result['matchs'].append({'home': int(game['teams']['away']['score']), 'away': int(game['teams']['home']['score'])})
                if game_away_id == away:
                    print(game['gameDate'], game['teams']['home']['score'], game['teams']['away']['score'])
                    if int(game['teams']['home']['score']) > int(game['teams']['away']['score']):
                        result['home_win'] = result['home_win'] + 1
                    elif int(game['teams']['home']['score']) < int(game['teams']['away']['score']):
                        result['away_win'] = result['away_win'] + 1
                    result['matchs'].append({'home': int(game['teams']['home']['score']), 'away': int(game['teams']['away']['score'])})
        else:
            print('No date in get matchup season')
        return result

    def get_matchup_result(self, matchup):
        # statuscode = {}
        # statuscode[1] = 'Scheduled'
        # statuscode[2] = 'Pre-Game'
        # statuscode[3] = 'In Progress'
        # statuscode[4] = 'In Progress - Critical'
        # statuscode[5] = 'Game Over'
        # statuscode[6] = 'Final'
        # statuscode[7] = 'Final'

        result = {}
        # home_id = matchup['home']
        away_id = matchup['away']
        home_win = 0
        away_win = 0
        for game in matchup['schedule']:
            game_home_id = game['teams']['home']['team']['id']
            game_away_id = game['teams']['away']['team']['id']
            if game['gameType'] == 'P':
                if game_home_id == away_id or game_away_id == away_id:
                    if game_home_id == away_id:  # reverse
                        away_score = game['teams']['home']['score']
                        home_score = game['teams']['away']['score']
                        away_shots = game['linescore']['teams']['home']['shotsOnGoal']
                        home_shots = game['linescore']['teams']['away']['shotsOnGoal']
                    else:
                        away_score = game['teams']['away']['score']
                        home_score = game['teams']['home']['score']
                        away_shots = game['linescore']['teams']['away']['shotsOnGoal']
                        home_shots = game['linescore']['teams']['home']['shotsOnGoal']
                    if int(game['status']['statusCode']) == 7:
                        if home_score > away_score:
                            home_win = home_win + 1
                        elif home_score < away_score:
                            away_win = away_win + 1
                    elif int(game['status']['statusCode']) in [3, 4, 5, 6]:
                        hi = self._teams[matchup['home']]
                        ai = self._teams[matchup['away']]
                        # period = game['linescore']['currentPeriod']
                        result = self.get_live_result(game['link'])
                        if game_home_id == away_id:
                            away_stats = result['liveData']['boxscore']['teams']['home']['teamStats']['teamSkaterStats']
                            home_stats = result['liveData']['boxscore']['teams']['away']['teamStats']['teamSkaterStats']
                        else:
                            away_stats = result['liveData']['boxscore']['teams']['away']['teamStats']['teamSkaterStats']
                            home_stats = result['liveData']['boxscore']['teams']['home']['teamStats']['teamSkaterStats']

                        period = game['linescore']['currentPeriodOrdinal']
                        rtime = game['linescore']['currentPeriodTimeRemaining']
                        print("Game {status} \033[0;94m{h}\033[0m {hsc}-{asc} \033[0;94m{a}\033[0m - {t} of {p} - Shots:\033[0;94m{h}\033[0m {hsh}-{ash} \033[0;94m{a}\033[0m - Faceoff:\033[0;94m{h}\033[0m {hf}-{af} \033[0;94m{a}\033[0m".format(hf=home_stats['faceOffWinPercentage'], af=away_stats['faceOffWinPercentage'], status=game['status']['detailedState'], h=hi['info']['abbreviation'], hsc=home_score, asc=away_score, a=ai['info']['abbreviation'], p=period, t=rtime, hsh=home_shots, ash=away_shots))
        result['home_win'] = home_win
        result['away_win'] = away_win
        return result

    def is_matchup_finished(self, matchup, victory=4):
        return matchup['result']['home_win'] == victory or matchup['result']['away_win'] == victory

    def get_matchup_winner(self, matchup, victory=4):
        if matchup['result']['home_win'] == victory:
            return matchup['home']
        if matchup['result']['away_win'] == victory:
            return matchup['away']
        return 0

    def update_matchups(self):
        ms = list(self._matchups.values())
        ms = sorted(ms, key=lambda k: k['round'])
        for matchup in ms:
            self.update_matchup(matchup)

    def update_matchup(self, matchup, home=0, away=0):
        if self.is_matchup_finished(matchup):
            return

        if matchup['home'] != 0 and matchup['away'] != 0:
            # update result and maybe pass to next stage
            matchup['schedule'] = self.get_matchup_schedule(matchup)
            if matchup['start'] == '':
                matchup['start'] = self.get_matchup_start(matchup)
            matchup['result'] = self.get_matchup_result(matchup)
            # if self.is_matchup_finished(matchup) and matchup['next'] is not None:
            #     print('Finished', matchup['id'])
            #     self.update_matchup(matchup['next'], self.get_matchup_winner(matchup))
        else:
            if matchup['home'] == 0:
                matchup['home'] = home
                matchup['away'] = away
            else:
                matchup['away'] = home

            if matchup['home'] != 0 and matchup['away'] != 0:
                # Begin matchup
                hi = self._teams[matchup['home']]
                ai = self._teams[matchup['away']]
                if int(hi['standings']['leagueRank']) > int(ai['standings']['leagueRank']):
                    matchup['home'] = ai['info']['id']
                    matchup['away'] = hi['info']['id']
                hi = self._teams[matchup['home']]
                ai = self._teams[matchup['away']]
                matchup['season'] = self.get_matchup_season_result(matchup['home'], matchup['away'])
                matchup['schedule'] = self.get_matchup_schedule(matchup)
                matchup['start'] = self.get_matchup_start(matchup)
                matchup['result'] = self.get_matchup_result(matchup)

    def update_matchups(self):
        ms = list(self._matchups.values())
        ms = sorted(ms, key=lambda k: k['round'])
        for matchup in ms:
            self.update_matchup(matchup)

    def get_schedule(self, team):
        # print('Get schedule for ' + str(team))
        url = 'https://statsapi.web.nhl.com/api/v1/schedule?startDate=' + str(self._year) + '-10-01&endDate=' + str(self._year + 1) + '-05-29&expand=schedule.teams,schedule.linescore,schedule.broadcasts,schedule.ticket,schedule.game.content.media.epg&leaderCategories=&site=en_nhlCA&teamId=' + str(team)
        team_schedule = requests.get(url)
        return team_schedule.json()

    def get_playoff_schedule(self, team):
        url = 'https://statsapi.web.nhl.com/api/v1/schedule?startDate=' + str(self._year + 1) + '-04-01&endDate=' + str(self._year + 1) + '-10-15&expand=schedule.teams,schedule.linescore,&site=en_nhlCA&teamId=' + str(team)
        # print(url)
        team_schedule = requests.get(url)
        return team_schedule.json()

    def fetch_data(self):
        data = requests.get('http://' + self._server + '/nhlplayoffs/api/v3.0/' + str(self._year) + '/data').json()
        return data

    def update_data(self, data):
        url = 'http://' + self._server + '/nhlplayoffs/api/v3.0/' + str(self._year) + '/data'
        headers = {'content-type': 'application/json'}
        requests.post(url, data=json.dumps(data), headers=headers)

    def build_matchup_tree(self, raw_matchups):
        matchups = {}
        for matchup_raw in list(raw_matchups.values()):
            matchup = matchup_raw.copy()
            matchups[matchup['id']] = matchup

        for matchup in list(matchups.values()):
            next = matchup['next']
            right = matchup['right']
            left = matchup['left']
            if next in raw_matchups:
                matchup['next'] = matchups[next]
            if right in raw_matchups:
                matchup['right'] = matchups[right]
            if left in raw_matchups:
                matchup['left'] = matchups[left]
        return matchups

    def store_matchup_tree(self, matchups):
        raw_matchups = {}
        for matchup in list(matchups.values()):
            raw_matchup = matchup.copy()
            if matchup['next'] is not None:
                raw_matchup['next'] = matchup['next']['id']
            if matchup['right'] is not None:
                raw_matchup['right'] = matchup['right']['id']
            if matchup['left'] is not None:
                raw_matchup['left'] = matchup['left']['id']
            raw_matchups[raw_matchup['id']] = raw_matchup
        return raw_matchups

    def load(self):
        data = self.fetch_data()
        self._teams = {}
        for m in data['teams'].items():
            self._teams[int(m[0])] = m[1]
        self._current_round = data['current_round']
        self._matchups = self.build_matchup_tree(data['matchups'])

    def store(self):
        data = {}
        data['teams'] = self._teams
        data['current_round'] = self._current_round
        data['matchups'] = self.store_matchup_tree(self._matchups)
        self.update_data(data)

    def display(self):
        nb_round = 4
        width = (nb_round * 2) - 1
        heigh = (2**(nb_round - 1)) - 1
        display = [['' for x in range(width)] for y in range(heigh)]

        def walk_matchup_tree(root, x, y, dx):
            display[x][y] = root['id']
            if root['left'] is not None:
                walk_matchup_tree(root['left'], x + dx, y - (root['round'] - 1), dx)
            if root['right'] is not None:
                walk_matchup_tree(root['right'], x + dx, y + (root['round'] - 1), dx)

        display[3][2] = 'sc'
        walk_matchup_tree(self._matchups['w'], 2, 3, -1)
        walk_matchup_tree(self._matchups['e'], 4, 3, 1)

        for y in range(7):
            for x in range(7):
                id = display[x][y]
                if id != '':
                    matchup = self._matchups[id]
                    if matchup['home'] == 0:
                        sys.stdout.write('{0:15}'.format(id))
                    else:
                        home = self._teams[matchup['home']]['info']['abbreviation']
                        away = '?'
                        if matchup['away'] != 0:
                            away = self._teams[matchup['away']]['info']['abbreviation']
                        sys.stdout.write('\033[0;94m{0:3}\033[0m-{2} vs {3}-\033[0;94m{1:3}\033[0m'.format(home, away, matchup['result']['home_win'], matchup['result']['away_win']))
                else:
                    sys.stdout.write('{0:15}'.format(id))
            sys.stdout.write('\n')


In [ ]:
upd = Updater('localhost:5000', 2020)
upd.update_matchups()

In [ ]:
upd.run()
upd.display()

In [8]:
teams = []
matchups = upd.get_matchup_round(1)
for matchup in matchups:
    if upd.is_matchup_finished(matchup):
        winner = upd.get_matchup_winner(matchup)
        teams.append(upd._teams[winner])

In [ ]:
c1 = upd._matchups['c1']
c = upd._matchups['c']
p = upd._matchups['p']
winner = upd.get_matchup_winner(c1)
if winner == c1["home"]:
    upd.update_matchup(c, winner)
else:
    upd.update_matchup(c, 0, winner)

In [ ]:
c1 = upd._matchups['c1']
c2 = upd._matchups['c2']
c = upd._matchups['c']
p = upd._matchups['p']
winner = upd.get_matchup_winner(c2)
if winner == c2["home"]:
    if c["home"] == 0:
        upd.update_matchup(c, winner)
    else:
        upd.update_matchup(p, winner)
else:
    upd.update_matchup(p, 0, winner)

In [ ]:
standing = upd.get_round_standings(1)

In [ ]:
c1 = upd._matchups['c1']
c2 = upd._matchups['c2']
p1 = upd._matchups['p1']
p2 = upd._matchups['p2']
c = upd._matchups['c']
p = upd._matchups['p']
a1 = upd._matchups['a1']
a2 = upd._matchups['a2']
m1 = upd._matchups['m1']
m2 = upd._matchups['m2']
a = upd._matchups['a']
m = upd._matchups['m']

In [ ]:
r1wnext = [(c, True), (p, True), (p,False), (c, False)]
r2w = [('w', True), ('w', False)]
r1enext = [(a, True), (m, True), (m,False), (a, False)]
r2e = [('e', True), ('e', False)]

In [ ]:
r1w = [c1, c2, p1, p2]
r1e = [a1, a2, m1, m2]

In [ ]:
def update_round_home(winner, rnext):
    for r in rnext:
        if r[1]:
            if r[0]["home"] == 0:
                print("Set Matchup home", r[0]['id'], r[0]['home'], r[0]['away'], winner)
                upd.update_matchup(r[0], winner)
                return
        else:
            if r[0]["away"] == 0:
                print("Set Matchup away", r[0]['id'], r[0]['home'], r[0]['away'], winner)
                upd.update_matchup(r[0], winner)
                return

def update_round_away(winner, rnext):
    rnr = rnext.copy()
    rnr.reverse()
    for r in rnr:
        if r[1]:
            if r[0]["home"] == 0:
                print("Set Matchup home", r[0]['id'], r[0]['home'], r[0]['away'], winner)
                upd.update_matchup(r[0], winner)
                return
        else:
            if r[0]["away"] == 0:
                print("Set Matchup away", r[0]['id'], r[0]['home'], r[0]['away'], winner)
                upd.update_matchup(r[0], winner)
                return

In [ ]:
for m in r1w:
    if upd.is_matchup_finished(m):
        winner = upd.get_matchup_winner(m)
        if winner == m['home']:
            print("Home winner", m["id"], winner)
            update_round_home(winner, r1wnext)
        else:
            print("Away winner", m["id"], upd._teams[winner]["info"]["abbreviation"])
            update_round_away(winner, r1wnext)
            
# for m in r1e:
#     if upd.is_matchup_finished(m):
#         winner = upd.get_matchup_winner(m)
#         if winner == m['home']:
#             print("Home winner", m["id"], winner)
#             update_round_home(winner, r1enext)
#         else:
#             print("Away winner", m["id"])
#             update_round_away(winner, r1enext)
            

In [ ]:
for r in r1:
    print(r)

In [ ]:
matchups = upd.get_matchup_round(2)

In [ ]:
for m in matchups:
    print(m['home'], m['away'])

In [ ]:
c["away"] = 0

In [ ]:
upd.store()

In [ ]:
t = [1,2, 3]

In [ ]:
upd._teams[2]

In [ ]:
t

In [ ]:
upd.update_matchup(c, 23)

In [ ]:
upd.store()

In [ ]:
c.keys()

In [ ]:
c["result"] = {'home_win': 0, 'away_win': 0}

In [ ]:
 matchup = {'id': id, 'home': 0, 'away': 0, 'round': round, 'start': '', 'result': {}, 'schedule': [], 'season': {}, 'next': next}
        matchup['left'] = None
        matchup['right'] = None
        matchup['result'] = {'home_win': 0, 'away_win': 0}

In [ ]:
teams = upd.get_teams()

In [ ]:
for tid in teams:
    team = teams[tid]
    print(team["info"]["division"], team["info"]["conference"])

In [4]:
class NHLQuery(object):

    def __init__(self, year):
        self._year = year
    
    def get_live_result(self, link):
        url = 'https://statsapi.web.nhl.com' + link
        live = requests.get(url).json()
        return live

    def get_team(self, id):
        url = 'https://statsapi.web.nhl.com/api/v1/teams/' + str(id)
        team = requests.get(url).json()
        return team['teams'][0]

    def get_teams(self):
        ystr = str(self._year) + str(self._year + 1)
        url = 'https://statsapi.web.nhl.com/api/v1/standings?season=' + ystr
        standings = requests.get(url).json()
        teams = {}
        for record in standings["records"]:
            for team in record['teamRecords']:
                info = self.get_team(team['team']['id'])
                team_record = {'info': info, 'standings': team, 'schedule': []}
                teams[team['team']['id']] = team_record
        return teams
    
    def get_schedule(self, team):
        # print('Get schedule for ' + str(team))
        url = 'https://statsapi.web.nhl.com/api/v1/schedule?startDate=' + str(self._year) + '-10-01&endDate=' + str(self._year + 1) + '-05-29&expand=schedule.teams,schedule.linescore,schedule.broadcasts,schedule.ticket,schedule.game.content.media.epg&leaderCategories=&site=en_nhlCA&teamId=' + str(team)
        team_schedule = requests.get(url)
        return team_schedule.json()

    def get_playoff_schedule(self, team):
        url = 'https://statsapi.web.nhl.com/api/v1/schedule?startDate=' + str(self._year + 1) + '-04-01&endDate=' + str(self._year + 1) + '-10-15&expand=schedule.teams,schedule.linescore,&site=en_nhlCA&teamId=' + str(team)
        # print(url)
        team_schedule = requests.get(url)
        return team_schedule.json()

In [5]:
class MatchupTree(object):

    def __init__(self):
        self._matchups = {}

    def get(self, matchup_id):
        return self._matchups.get(matchup_id, [])
    
    def __getitem__(self, key):
        return self.get(key)
    
    def __setitem__(self, key, value):
        self._matchups[key] = value
    
    def __iter__(self):
        return self._matchups.__iter__()
    
    def __contains__(self, key):
        return key in self._db

    def __len__(self):
        return len(self.matchups)

    @property
    def matchups(self):
        return list(self._matchups.values())

    def restore(self, raw_matchups):
        matchups = {}
        for matchup_raw in list(raw_matchups.values()):
            matchup = matchup_raw.copy()
            matchups[matchup['id']] = matchup

        for matchup in list(matchups.values()):
            next = matchup['next']
            right = matchup['right']
            left = matchup['left']
            if next in raw_matchups:
                matchup['next'] = matchups[next]
            if right in raw_matchups:
                matchup['right'] = matchups[right]
            if left in raw_matchups:
                matchup['left'] = matchups[left]
        return matchups

    def store(self):
        raw_matchups = {}
        for matchup in list(self._matchups.values()):
            raw_matchup = matchup.copy()
            if matchup['next'] is not None:
                raw_matchup['next'] = matchup['next']['id']
            if matchup['right'] is not None:
                raw_matchup['right'] = matchup['right']['id']
            if matchup['left'] is not None:
                raw_matchup['left'] = matchup['left']['id']
            raw_matchups[raw_matchup['id']] = raw_matchup
        return raw_matchups
    
    def set_matchup_childs(self, matchup, right, left):
        matchup['left'] = left
        matchup['right'] = right

    def create_matchup(self, id, round, next):
        matchup = {'id': id, 'home': 0, 'away': 0, 'round': round, 'start': '', 'result': {}, 'schedule': [], 'season': {}, 'next': next}
        matchup['left'] = None
        matchup['right'] = None
        matchup['result'] = {'home_win': 0, 'away_win': 0}
        return matchup
    
    def is_matchup_finished(self, matchup, victory=4):
        return matchup['result']['home_win'] == victory or matchup['result']['away_win'] == victory

    def get_matchup_winner(self, matchup, victory=4):
        if matchup['result']['home_win'] == victory:
            return matchup['home']
        if matchup['result']['away_win'] == victory:
            return matchup['away']
        return 0

    def get_matchup_round(self, round):
        matchups = []
        for matchup in self._matchups.values():
            if matchup['round'] == round:
                matchups.append(matchup)
        return matchups
    
    def get_matchup_start(self, matchup):
        if len(matchup['schedule']) == 0:
            return ''
        return matchup['schedule'][0]['gameDate']

In [6]:
class Updater(object):

    def __init__(self, server, year):
        self._server = server
        self._year = year
        self._season_games = 82
        self._current_round = 0
        self._auto_move = False
        self._teams = {}
        self._standings = {}
        self._query = NHLQuery(self._year)
        self._matchups = MatchupTree()
        self.load()
    
    @property
    def teams(self):
        return list(self._teams.values())

    def load(self):
        data = self.fetch_data()
        self._teams = {}
        for m in data['teams'].items():
            self._teams[int(m[0])] = m[1]
        self._current_round = data['current_round']
        self._matchups.restore(data['matchups'])

    def store(self):
        data = {}
        data['teams'] = self._teams
        data['current_round'] = self._current_round
        data['matchups'] = self._matchups.store()
        self.update_data(data)

    def fetch_data(self):
        data = requests.get('http://' + self._server + '/nhlplayoffs/api/v3.0/' + str(self._year) + '/data').json()
        return data

    def update_data(self, data):
        url = 'http://' + self._server + '/nhlplayoffs/api/v3.0/' + str(self._year) + '/data'
        headers = {'content-type': 'application/json'}
        requests.post(url, data=json.dumps(data), headers=headers)
    
    def display(self):
        nb_round = 4
        width = (nb_round * 2) - 1
        heigh = (2**(nb_round - 1)) - 1
        display = [['' for x in range(width)] for y in range(heigh)]

        def walk_matchup_tree(root, x, y, dx):
            display[x][y] = root['id']
            if root['left'] is not None:
                walk_matchup_tree(root['left'], x + dx, y - (root['round'] - 1), dx)
            if root['right'] is not None:
                walk_matchup_tree(root['right'], x + dx, y + (root['round'] - 1), dx)

        display[3][2] = 'sc'
        walk_matchup_tree(self._matchups['sc1'], 2, 3, -1)
        walk_matchup_tree(self._matchups['sc2'], 4, 3, 1)

        for y in range(7):
            for x in range(7):
                id = display[x][y]
                if id != '':
                    matchup = self._matchups[id]
                    if matchup['home'] == 0:
                        sys.stdout.write('{0:15}'.format(id))
                    else:
                        home = self._teams[matchup['home']]['info']['abbreviation']
                        away = '?'
                        if matchup['away'] != 0:
                            away = self._teams[matchup['away']]['info']['abbreviation']
                        sys.stdout.write('\033[0;94m{0:3}\033[0m-{2} vs {3}-\033[0;94m{1:3}\033[0m'.format(home, away, matchup['result']['home_win'], matchup['result']['away_win']))
                else:
                    sys.stdout.write('{0:15}'.format(id))
            sys.stdout.write('\n')
    
    def is_season_finished(self):
        for team in self.teams:
            remaining = self._season_games - team['standings']['gamesPlayed']
            if remaining > 0:
                return False
        return True
    
    def is_round_finished(self, round):
        matchups = self._matchups.get_matchup_round(round)
        for matchup in matchups:
            if not self._matchups.is_matchup_finished(matchup):
                return False
        return True

    def get_season_standings(self):
        return self.get_standings(self.teams)
    
    def get_round_standings(self, round):
        teams = []
        matchups = self._matchups.get_matchup_round(1)
        for matchup in matchups:
            teams.append(upd._teams[matchup['home']])
            teams.append(upd._teams[matchup['away']])
        return self.get_standings(teams)

    def get_round_winners_standings(self, round):
        teams = []
        matchups = self._matchups.get_matchup_round(round)
        for matchup in matchups:
            winner = self._matchups.get_matchup_winner(matchup)
            teams.append(self._teams[winner])
        return self.get_standings(teams)

    def get_matchup_schedule(self, matchup, schedules=None):
        home_id = matchup['home']
        away_id = matchup['away']
        result = []
        s = schedules
        if schedules is None:
            s = self._query.get_playoff_schedule(int(home_id))
        if 'dates' in s:
            for date in s['dates']:
                game = date['games'][0]
                game_home_id = game['teams']['home']['team']['id']
                game_away_id = game['teams']['away']['team']['id']
                if game['gameType'] == 'P':
                    if game_home_id == away_id or game_away_id == away_id:
                        result.append(game)
        else:
            print('No date in get matchup')
        result = sorted(result, key=lambda k: self.parse_time(k['gameDate']))
        return result

    def parse_time(self, timestamp):
        from_zone = tz.gettz('UTC')
        to_zone = tz.gettz('America/New_York')
        utc = datetime.strptime(timestamp, '%Y-%m-%dT%H:%M:%SZ')
        utc = utc.replace(tzinfo=from_zone)
        return utc.astimezone(to_zone)

    def update_round_home(self, winner, rnext):
        for r in rnext:
            if r[1]:
                if r[0]["home"] == 0:
                    print("Set Matchup home", r[0]['id'], r[0]['home'], r[0]['away'], winner)
                    self.update_matchup(r[0], winner)
                    return
            else:
                if r[0]["away"] == 0:
                    print("Set Matchup away", r[0]['id'], r[0]['home'], r[0]['away'], winner)
                    self.update_matchup(r[0], winner)
                    return

    def update_round_away(self, winner, rnext):
        rnr = rnext.copy()
        rnr.reverse()
        for r in rnr:
            if r[1]:
                if r[0]["home"] == 0:
                    print("Set Matchup home", r[0]['id'], r[0]['home'], r[0]['away'], winner)
                    self.update_matchup(r[0], winner)
                    return
            else:
                if r[0]["away"] == 0:
                    print("Set Matchup away", r[0]['id'], r[0]['home'], r[0]['away'], winner)
                    self.update_matchup(r[0], winner)
                    return

    

    def get_matchup_season_result(self, home, away):
        result = {'home_win': 0, 'away_win': 0, 'matchs': []}
        schedule = self._teams[home]['schedule']
        if len(schedule) == 0:
            schedule = self._query.get_schedule(home)
            # self._teams[home]['schedule'] = schedule

        if 'dates' in schedule:
            for date in schedule['dates']:
                game = date['games'][0]
                game_home_id = game['teams']['home']['team']['id']
                game_away_id = game['teams']['away']['team']['id']
                if game_home_id == away:
                    print(game['gameDate'], game['teams']['away']['score'], game['teams']['home']['score'])
                    if int(game['teams']['home']['score']) > int(game['teams']['away']['score']):
                        result['away_win'] = result['away_win'] + 1
                    elif int(game['teams']['home']['score']) < int(game['teams']['away']['score']):
                        result['home_win'] = result['home_win'] + 1
                    result['matchs'].append({'home': int(game['teams']['away']['score']), 'away': int(game['teams']['home']['score'])})
                if game_away_id == away:
                    print(game['gameDate'], game['teams']['home']['score'], game['teams']['away']['score'])
                    if int(game['teams']['home']['score']) > int(game['teams']['away']['score']):
                        result['home_win'] = result['home_win'] + 1
                    elif int(game['teams']['home']['score']) < int(game['teams']['away']['score']):
                        result['away_win'] = result['away_win'] + 1
                    result['matchs'].append({'home': int(game['teams']['home']['score']), 'away': int(game['teams']['away']['score'])})
        else:
            print('No date in get matchup season')
        return result

    def get_matchup_result(self, matchup):
        # statuscode = {}
        # statuscode[1] = 'Scheduled'
        # statuscode[2] = 'Pre-Game'
        # statuscode[3] = 'In Progress'
        # statuscode[4] = 'In Progress - Critical'
        # statuscode[5] = 'Game Over'
        # statuscode[6] = 'Final'
        # statuscode[7] = 'Final'

        result = {}
        # home_id = matchup['home']
        away_id = matchup['away']
        home_win = 0
        away_win = 0
        for game in matchup['schedule']:
            game_home_id = game['teams']['home']['team']['id']
            game_away_id = game['teams']['away']['team']['id']
            if game['gameType'] == 'P':
                if game_home_id == away_id or game_away_id == away_id:
                    if game_home_id == away_id:  # reverse
                        away_score = game['teams']['home']['score']
                        home_score = game['teams']['away']['score']
                        away_shots = game['linescore']['teams']['home']['shotsOnGoal']
                        home_shots = game['linescore']['teams']['away']['shotsOnGoal']
                    else:
                        away_score = game['teams']['away']['score']
                        home_score = game['teams']['home']['score']
                        away_shots = game['linescore']['teams']['away']['shotsOnGoal']
                        home_shots = game['linescore']['teams']['home']['shotsOnGoal']
                    if int(game['status']['statusCode']) == 7:
                        if home_score > away_score:
                            home_win = home_win + 1
                        elif home_score < away_score:
                            away_win = away_win + 1
                    elif int(game['status']['statusCode']) in [3, 4, 5, 6]:
                        hi = self._teams[matchup['home']]
                        ai = self._teams[matchup['away']]
                        # period = game['linescore']['currentPeriod']
                        result = self._query.get_live_result(game['link'])
                        if game_home_id == away_id:
                            away_stats = result['liveData']['boxscore']['teams']['home']['teamStats']['teamSkaterStats']
                            home_stats = result['liveData']['boxscore']['teams']['away']['teamStats']['teamSkaterStats']
                        else:
                            away_stats = result['liveData']['boxscore']['teams']['away']['teamStats']['teamSkaterStats']
                            home_stats = result['liveData']['boxscore']['teams']['home']['teamStats']['teamSkaterStats']

                        period = game['linescore']['currentPeriodOrdinal']
                        rtime = game['linescore']['currentPeriodTimeRemaining']
                        print("Game {status} \033[0;94m{h}\033[0m {hsc}-{asc} \033[0;94m{a}\033[0m - {t} of {p} - Shots:\033[0;94m{h}\033[0m {hsh}-{ash} \033[0;94m{a}\033[0m - Faceoff:\033[0;94m{h}\033[0m {hf}-{af} \033[0;94m{a}\033[0m".format(hf=home_stats['faceOffWinPercentage'], af=away_stats['faceOffWinPercentage'], status=game['status']['detailedState'], h=hi['info']['abbreviation'], hsc=home_score, asc=away_score, a=ai['info']['abbreviation'], p=period, t=rtime, hsh=home_shots, ash=away_shots))
        result['home_win'] = home_win
        result['away_win'] = away_win
        return result

    def update_matchups(self):
        ms = self._matchups.matchups
        ms = sorted(ms, key=lambda k: k['round'])
        for matchup in ms:
            self.update_matchup(matchup)

    def update_matchup(self, matchup, home=0, away=0):
        if self._matchups.is_matchup_finished(matchup):
            return
        if matchup['home'] != 0 and matchup['away'] != 0:
            # update result and maybe pass to next stage
            matchup['schedule'] = self.get_matchup_schedule(matchup)
            if matchup['start'] == '':
                matchup['start'] = self._matchups.get_matchup_start(matchup)
            matchup['result'] = self.get_matchup_result(matchup)
            if self._auto_move:
                if self.is_matchup_finished(matchup) and matchup['next'] is not None:
                    print('Finished', matchup['id'])
                    self.update_matchup(matchup['next'], self.get_matchup_winner(matchup))
        else:
            if matchup['home'] == 0:
                matchup['home'] = home
                matchup['away'] = away
            else:
                matchup['away'] = home

            if matchup['home'] != 0 and matchup['away'] != 0:
                # Begin matchup
                hi = self._teams[matchup['home']]
                ai = self._teams[matchup['away']]
                if int(hi['standings']['leagueRank']) > int(ai['standings']['leagueRank']):
                    matchup['home'] = ai['info']['id']
                    matchup['away'] = hi['info']['id']
                hi = self._teams[matchup['home']]
                ai = self._teams[matchup['away']]
                matchup['season'] = self.get_matchup_season_result(matchup['home'], matchup['away'])
                matchup['schedule'] = self.get_matchup_schedule(matchup)
                matchup['start'] = self._matchups.get_matchup_start(matchup)
                matchup['result'] = self.get_matchup_result(matchup)

    def create_matchups(self, standings, round):
        if round == 0:
            self.create_matchups_round0(standings)
        elif round == 1:
            self.create_matchups_round1(standings)
        elif round == 2:
            self.create_matchups_round2(standings)
        elif round == 3:
            self.create_matchups_round3(standings)

    def create_matchups_round0(self, standings):
        pass

    def create_matchups_round1(self, standings):
        pass

    def create_matchups_round2(self, standings):
        pass

    def create_matchups_round3(self, standings):
        pass
    
    def run(self):
        pass

    def create_matchups_tree(self):
        pass
    
    def get_standings(self, teams):
        pass

    

In [7]:
class Updater2021(Updater):

    def __init__(self, server):
        super(Updater2021, self).__init__(server, 2020)
        self._season_games = 56
    
    def run(self):
        if self._current_round == 0:
            self._teams = self._query.get_teams()
            self.create_matchups_tree()

            if len(self.teams) > 0:
                self._standings = self.get_season_standings()
                self.create_matchups(self._standings, 0)
                if self.is_season_finished():
                    self._current_round += 1
                    self.store()
        else:
            self.update_matchups()
            if self.is_round_finished(self._current_round):
                self.create_matchups(self.get_round_winners_standings(self._current_round), self._current_round)
                self._current_round += 1
            self.store()
    
    def create_matchups_tree(self):
        sc = self._matchups.create_matchup('sc', 4, None)
        self._matchups[sc['id']] = sc

        sc1 = self._matchups.create_matchup('sc1', 3, sc)
        sc2 = self._matchups.create_matchup('sc2', 3, sc)
        self._matchups[sc1['id']] = sc1
        self._matchups[sc2['id']] = sc2

        n = self._matchups.create_matchup('n', 2, sc1)
        c = self._matchups.create_matchup('c', 2, sc1)
        e = self._matchups.create_matchup('e', 2, sc2)
        w = self._matchups.create_matchup('w', 2, sc2)
        self._matchups[n['id']] = n
        self._matchups[c['id']] = c
        self._matchups[e['id']] = e
        self._matchups[w['id']] = w

        n1 = self._matchups.create_matchup('n1', 1, n)
        n2 = self._matchups.create_matchup('n2', 1, n)
        c1 = self._matchups.create_matchup('c1', 1, c)
        c2 = self._matchups.create_matchup('c2', 1, c)
        e1 = self._matchups.create_matchup('e1', 1, e)
        e2 = self._matchups.create_matchup('e2', 1, e)
        w1 = self._matchups.create_matchup('w1', 1, w)
        w2 = self._matchups.create_matchup('w2', 1, w)
        self._matchups[n1['id']] = n1
        self._matchups[n2['id']] = n2
        self._matchups[c1['id']] = c1
        self._matchups[c2['id']] = c2
        self._matchups[e1['id']] = e1
        self._matchups[e2['id']] = e2
        self._matchups[w1['id']] = w1
        self._matchups[w2['id']] = w2

        # build tree
        self._matchups.set_matchup_childs(sc, sc1, sc2)

        self._matchups.set_matchup_childs(sc1, n, c)
        self._matchups.set_matchup_childs(sc2, e, w)

        self._matchups.set_matchup_childs(n, n2, n1)
        self._matchups.set_matchup_childs(c, c2, c1)
        self._matchups.set_matchup_childs(e, e2, e1)
        self._matchups.set_matchup_childs(w, w2, w1)
    
    def get_standings(self, teams):
        standings = {'Scotia North': [], 'Discover Central': [], 'MassMutual East': [], 'Honda West': [], 'teams': []}

        league = sorted(teams, key=lambda k: int(k['standings']['leagueRank']))
        for team in league:
            standings['teams'].append(team)
            standings[team['info']['division']['name']].append(team)
        standings['teams'] = sorted(standings['teams'], key=lambda k: int(k['standings']['leagueRank']))

        standings['Scotia North'] = sorted(standings['Scotia North'], key=lambda k: int(k['standings']['divisionRank']))
        standings['Discover Central'] = sorted(standings['Discover Central'], key=lambda k: int(k['standings']['divisionRank']))
        standings['MassMutual East'] = sorted(standings['MassMutual East'], key=lambda k: int(k['standings']['divisionRank']))
        standings['Honda West'] = sorted(standings['Honda West'], key=lambda k: int(k['standings']['divisionRank']))
        return standings
    
    def create_matchups_round0(self, standings):
        n1 = self._matchups['n1']
        n2 = self._matchups['n2']
        c1 = self._matchups['c1']
        c2 = self._matchups['c2']
        self.update_matchup(n1, standings['Scotia North'][0]['info']['id'], standings['Scotia North'][3]['info']['id'])
        self.update_matchup(n2, standings['Scotia North'][1]['info']['id'], standings['Scotia North'][2]['info']['id'])
        self.update_matchup(c1, standings['Discover Central'][0]['info']['id'], standings['Discover Central'][3]['info']['id'])
        self.update_matchup(c2, standings['Discover Central'][1]['info']['id'], standings['Discover Central'][2]['info']['id'])
        e1 = self._matchups['e1']
        e2 = self._matchups['e2']
        w1 = self._matchups['w1']
        w2 = self._matchups['w2']
        self.update_matchup(e1, standings['MassMutual East'][0]['info']['id'], standings['MassMutual East'][3]['info']['id'])
        self.update_matchup(e2, standings['MassMutual East'][1]['info']['id'], standings['MassMutual East'][2]['info']['id'])
        self.update_matchup(w1, standings['Honda West'][0]['info']['id'], standings['Honda West'][3]['info']['id'])
        self.update_matchup(w2, standings['Honda West'][1]['info']['id'], standings['Honda West'][2]['info']['id'])

    def create_matchups_round1(self, standings):
        n = self._matchups['n']
        c = self._matchups['c']
        if len(standings['Scotia North']) == 2:
            self.update_matchup(n, standings['Scotia North'][0]['info']['id'], standings['Scotia North'][1]['info']['id'])
        if len(standings['Discover Central']) == 2:
            self.update_matchup(c, standings['Discover Central'][0]['info']['id'], standings['Discover Central'][1]['info']['id'])
        e = self._matchups['e']
        w = self._matchups['w']
        if len(standings['MassMutual East']) == 2:
            self.update_matchup(e, standings['MassMutual East'][0]['info']['id'], standings['MassMutual East'][1]['info']['id'])
        if len(standings['Honda West']) == 2:
            self.update_matchup(w, standings['Honda West'][0]['info']['id'], standings['Honda West'][1]['info']['id'])

    def create_matchups_round2(self, standings):
        sc1 = self._matchups['sc1']
        self.update_matchup(sc1, standings['teams'][0]['info']['id'], standings['teams'][3]['info']['id'])
        sc2 = self._matchups['sc2']
        self.update_matchup(sc2, standings['teams'][1]['info']['id'], standings['teams'][2]['info']['id'])

    def create_matchups_round3(self, standings):
        sc = self._matchups['sc']
        self.update_matchup(sc, standings['teams'][0]['info']['id'], standings['teams'][1]['info']['id'])
    


In [ ]:
upd = Updater2021('localhost:5000')

In [ ]:
upd.run()

In [ ]:
upd.display()

In [ ]:
upd.display()

In [ ]:
standings = {'Scotia North': [], 'Discover Central': [], 'MassMutual East': [], 'Honda West': [], 'teams': []}

league = sorted(list(upd._teams.values()), key=lambda k: int(k['standings']['leagueRank']))
for team in league:
    standings['teams'].append(team)
    standings[team['info']['division']['name']].append(team)
standings['teams'] = sorted(standings['teams'], key=lambda k: int(k['standings']['leagueRank']))

standings['Scotia North'] = sorted(standings['Scotia North'], key=lambda k: int(k['standings']['divisionRank']))
standings['Discover Central'] = sorted(standings['Discover Central'], key=lambda k: int(k['standings']['divisionRank']))
standings['MassMutual East'] = sorted(standings['MassMutual East'], key=lambda k: int(k['standings']['divisionRank']))
standings['Honda West'] = sorted(standings['Honda West'], key=lambda k: int(k['standings']['divisionRank']))

In [ ]:
teams = []
matchups = upd.get_matchup_round(1)
for matchup in matchups:
    teams.append(upd._teams[matchup['home']])
    teams.append(upd._teams[matchup['away']])
standings = {'Scotia North': [], 'Discover Central': [], 'MassMutual East': [], 'Honda West': [], 'teams': []}
league = sorted(teams, key=lambda k: int(k['standings']['leagueRank']))
for team in league:
    standings['teams'].append(team)
    standings[team['info']['division']['name']].append(team)
standings['teams'] = sorted(standings['teams'], key=lambda k: int(k['standings']['leagueRank']))

standings['Scotia North'] = sorted(standings['Scotia North'], key=lambda k: int(k['standings']['divisionRank']))
standings['Discover Central'] = sorted(standings['Discover Central'], key=lambda k: int(k['standings']['divisionRank']))
standings['MassMutual East'] = sorted(standings['MassMutual East'], key=lambda k: int(k['standings']['divisionRank']))
standings['Honda West'] = sorted(standings['Honda West'], key=lambda k: int(k['standings']['divisionRank']))

In [ ]:
dir(Updater2021)

In [ ]:
dir(Updater2021bak)

In [ ]:
dir(MatchupTree)

In [8]:
class Updater2022(Updater):

    def __init__(self, server):
        super(Updater2022, self).__init__(server, 2021)
        self._auto_move = True
    
    def run(self):
        if self._current_round == 0:
            self._teams = self._query.get_teams()
            self.create_matchups_tree()

            if len(self.teams) > 0:
                self._standings = self.get_season_standings()
                self.create_matchups(self._standings, 0)
                if self.is_season_finished():
                    self._current_round += 1
            #         self.store()
        else:
            self.update_matchups()
            # if self.is_round_finished(self._current_round):
            #     self.create_matchups(self.get_round_winners_standings(self._current_round), self._current_round)
            #     self._current_round += 1
            # self.store()
    
    def create_matchups_tree(self):
        sc = self._matchups.create_matchup('sc', 4, None)
        self._matchups[sc['id']] = sc

        sc1 = self._matchups.create_matchup('sc1', 3, sc)
        sc2 = self._matchups.create_matchup('sc2', 3, sc)
        self._matchups[sc1['id']] = sc1
        self._matchups[sc2['id']] = sc2

        a = self._matchups.create_matchup('a', 2, sc2)
        m = self._matchups.create_matchup('m', 2, sc2)
        c = self._matchups.create_matchup('c', 2, sc1)
        p = self._matchups.create_matchup('p', 2, sc1)
        self._matchups[a['id']] = a
        self._matchups[m['id']] = m
        self._matchups[c['id']] = c
        self._matchups[p['id']] = p

        a1 = self._matchups.create_matchup('a1', 1, a)
        a2 = self._matchups.create_matchup('a2', 1, a)
        m1 = self._matchups.create_matchup('m1', 1, m)
        m2 = self._matchups.create_matchup('m2', 1, m)
        c1 = self._matchups.create_matchup('c1', 1, c)
        c2 = self._matchups.create_matchup('c2', 1, c)
        p1 = self._matchups.create_matchup('p1', 1, p)
        p2 = self._matchups.create_matchup('p2', 1, p)
        self._matchups[a1['id']] = a1
        self._matchups[a2['id']] = a2
        self._matchups[m1['id']] = m1
        self._matchups[m2['id']] = m2
        self._matchups[c1['id']] = c1
        self._matchups[c2['id']] = c2
        self._matchups[p1['id']] = p1
        self._matchups[p2['id']] = p2

        # build tree
        self._matchups.set_matchup_childs(sc, sc1, sc2)

        self._matchups.set_matchup_childs(sc2, a, m)
        self._matchups.set_matchup_childs(sc1, c, p)

        self._matchups.set_matchup_childs(a, a2, a1)
        self._matchups.set_matchup_childs(m, m2, m1)
        self._matchups.set_matchup_childs(c, c2, c1)
        self._matchups.set_matchup_childs(p, p2, p1)
    
    def get_standings(self, teams):
        standings = {'Eastern': {'Atlantic': [], 'Metropolitan': [], 'teams': []}, 'Western': {'Central': [], 'Pacific': [], 'teams': []}, 'teams': []}

        league = sorted(list(upd._teams.values()), key=lambda k: int(k['standings']['divisionRank']))
        for team in league:
            standings['teams'].append(team)
            standings[team['info']['conference']['name']]['teams'].append(team)
            standings[team['info']['conference']['name']][team['info']['division']['name']].append(team)
        standings['teams'] = sorted(standings['teams'], key=lambda k: int(k['standings']['leagueRank']))

        standings['Eastern']['teams'] = sorted(standings['Eastern']['teams'], key=lambda k: int(k['standings']['conferenceRank']))
        standings['Western']['teams'] = sorted(standings['Western']['teams'], key=lambda k: int(k['standings']['conferenceRank']))

        standings['Eastern']['Atlantic'] = sorted(standings['Eastern']['Atlantic'], key=lambda k: int(k['standings']['divisionRank']))
        standings['Eastern']['Metropolitan'] = sorted(standings['Eastern']['Metropolitan'], key=lambda k: int(k['standings']['divisionRank']))
        standings['Western']['Central'] = sorted(standings['Western']['Central'], key=lambda k: int(k['standings']['divisionRank']))
        standings['Western']['Pacific'] = sorted(standings['Western']['Pacific'], key=lambda k: int(k['standings']['divisionRank']))
        return standings
    
    def create_matchups_round0(self, standings):
        ealeader = standings['Eastern']['Atlantic'][0]
        emleader = standings['Eastern']['Metropolitan'][0]
        wcleader = standings['Western']['Central'][0]
        wpleader = standings['Western']['Pacific'][0]
        for team in standings['Eastern']['teams']:
            if int(team['standings']['wildCardRank']) == 1:
                e1wild = team
            if int(team['standings']['wildCardRank']) == 2:
                e2wild = team

        for team in standings['Western']['teams']:
            if int(team['standings']['wildCardRank']) == 1:
                w1wild = team
            if int(team['standings']['wildCardRank']) == 2:
                w2wild = team

        if int(ealeader['standings']['conferenceRank']) < int(emleader['standings']['conferenceRank']):
            a1_teams = (ealeader, e2wild)
            m1_teams = (emleader, e1wild)
        else:
            a1_teams = (ealeader, e1wild)
            m1_teams = (emleader, e2wild)
        a2_teams = (standings['Eastern']['Atlantic'][1], standings['Eastern']['Atlantic'][2])
        m2_teams = (standings['Eastern']['Metropolitan'][1], standings['Eastern']['Metropolitan'][2])

        if int(wcleader['standings']['conferenceRank']) < int(wpleader['standings']['conferenceRank']):
            c1_teams = (wcleader, w2wild)
            p1_teams = (wpleader, w1wild)
        else:
            c1_teams = (wcleader, w1wild)
            p1_teams = (wpleader, w2wild)
        c2_teams = (standings['Western']['Central'][1], standings['Western']['Central'][2])
        p2_teams = (standings['Western']['Pacific'][1], standings['Western']['Pacific'][2])

        a1 = self._matchups['a1']
        a2 = self._matchups['a2']
        m1 = self._matchups['m1']
        m2 = self._matchups['m2']
        self.update_matchup(a1, a1_teams[0]['info']['id'], a1_teams[1]['info']['id'])
        self.update_matchup(a2, a2_teams[0]['info']['id'], a2_teams[1]['info']['id'])
        self.update_matchup(m1, m1_teams[0]['info']['id'], m1_teams[1]['info']['id'])
        self.update_matchup(m2, m2_teams[0]['info']['id'], m2_teams[1]['info']['id'])
        c1 = self._matchups['c1']
        c2 = self._matchups['c2']
        p1 = self._matchups['p1']
        p2 = self._matchups['p2']
        self.update_matchup(c1, c1_teams[0]['info']['id'], c1_teams[1]['info']['id'])
        self.update_matchup(c2, c2_teams[0]['info']['id'], c2_teams[1]['info']['id'])
        self.update_matchup(p1, p1_teams[0]['info']['id'], p1_teams[1]['info']['id'])
        self.update_matchup(p2, p2_teams[0]['info']['id'], p2_teams[1]['info']['id'])


In [10]:
upd = Updater2022('localhost:5000')

In [11]:
upd.run()

2021-10-27T23:00:00Z 4 1
2021-10-30T23:00:00Z 2 3
2022-04-26T23:00:00Z 0 0
2021-11-04T23:00:00Z 1 2
2021-12-10T00:00:00Z 5 3
2022-04-04T23:30:00Z 0 0
2022-04-22T00:00:00Z 0 0
2021-11-28T18:00:00Z 2 4
2022-03-04T00:00:00Z 0 4
2022-03-18T23:00:00Z 0 0
2022-03-28T23:00:00Z 0 0
2022-02-26T20:00:00Z 1 0
2022-03-25T23:00:00Z 0 0
2022-03-29T23:00:00Z 0 0
2022-04-07T23:00:00Z 0 0
2021-10-06T01:00:00Z 4 7
2021-10-27T00:00:00Z 1 3
2022-02-17T03:00:00Z 2 0
2022-02-27T03:00:00Z 3 2
2021-10-07T00:00:00Z 3 4
2022-01-02T00:00:00Z 6 4
2022-04-09T00:00:00Z 0 0
2022-04-16T19:00:00Z 0 0
2021-11-03T01:00:00Z 2 3
2022-04-20T00:00:00Z 0 0
2022-04-27T00:00:00Z 0 0
2021-12-06T01:00:00Z 5 1
2022-02-16T03:30:00Z 2 5
2022-03-31T01:30:00Z 0 0
2022-04-08T02:30:00Z 0 0


In [30]:
standings = {'Eastern': {'Atlantic': [], 'Metropolitan': [], 'teams': []}, 'Western': {'Central': [], 'Pacific': [], 'teams': []}, 'teams': []}

league = sorted(list(upd._teams.values()), key=lambda k: int(k['standings']['divisionRank']))
for team in league:
    standings['teams'].append(team)
    standings[team['info']['conference']['name']]['teams'].append(team)
    standings[team['info']['conference']['name']][team['info']['division']['name']].append(team)
standings['teams'] = sorted(standings['teams'], key=lambda k: int(k['standings']['leagueRank']))

standings['Eastern']['teams'] = sorted(standings['Eastern']['teams'], key=lambda k: int(k['standings']['conferenceRank']))
standings['Western']['teams'] = sorted(standings['Western']['teams'], key=lambda k: int(k['standings']['conferenceRank']))

standings['Eastern']['Atlantic'] = sorted(standings['Eastern']['Atlantic'], key=lambda k: int(k['standings']['divisionRank']))
standings['Eastern']['Metropolitan'] = sorted(standings['Eastern']['Metropolitan'], key=lambda k: int(k['standings']['divisionRank']))
standings['Western']['Central'] = sorted(standings['Western']['Central'], key=lambda k: int(k['standings']['divisionRank']))
standings['Western']['Pacific'] = sorted(standings['Western']['Pacific'], key=lambda k: int(k['standings']['divisionRank']))

In [34]:
ealeader = standings['Eastern']['Atlantic'][0]
emleader = standings['Eastern']['Metropolitan'][0]
wcleader = standings['Western']['Central'][0]
wpleader = standings['Western']['Pacific'][0]
for team in standings['Eastern']['teams']:
    if int(team['standings']['wildCardRank']) == 1:
        e1wild = team
    if int(team['standings']['wildCardRank']) == 2:
        e2wild = team

for team in standings['Western']['teams']:
    if int(team['standings']['wildCardRank']) == 1:
        w1wild = team
    if int(team['standings']['wildCardRank']) == 2:
        w2wild = team

if int(ealeader['standings']['conferenceRank']) < int(emleader['standings']['conferenceRank']):
    a1_teams = (ealeader, e2wild)
    m1_teams = (emleader, e1wild)
else:
    a1_teams = (ealeader, e1wild)
    m1_teams = (emleader, e2wild)
a2_teams = (standings['Eastern']['Atlantic'][1], standings['Eastern']['Atlantic'][2])
m2_teams = (standings['Eastern']['Metropolitan'][1], standings['Eastern']['Metropolitan'][2])

if int(wcleader['standings']['conferenceRank']) < int(wpleader['standings']['conferenceRank']):
    c1_teams = (wcleader, w2wild)
    p1_teams = (wpleader, w1wild)
else:
    c1_teams = (wcleader, w1wild)
    p1_teams = (wpleader, w2wild)
c2_teams = (standings['Western']['Central'][1], standings['Western']['Central'][2])
p2_teams = (standings['Western']['Pacific'][1], standings['Western']['Pacific'][2])

print(f"a1 {a1_teams[0]['info']['name']} {a1_teams[1]['info']['name']}")
print(f"m1 {m1_teams[0]['info']['name']} {m1_teams[1]['info']['name']}")
print(f"a2 {a2_teams[0]['info']['name']} {a2_teams[1]['info']['name']}")
print(f"m2 {m2_teams[0]['info']['name']} {m2_teams[1]['info']['name']}")

print(f"c1 {c1_teams[0]['info']['name']} {c1_teams[1]['info']['name']}")
print(f"p1 {p1_teams[0]['info']['name']} {p1_teams[1]['info']['name']}")
print(f"c2 {c2_teams[0]['info']['name']} {c2_teams[1]['info']['name']}")
print(f"p2 {p2_teams[0]['info']['name']} {p2_teams[1]['info']['name']}")


a1 Florida Panthers Boston Bruins
m1 Carolina Hurricanes Washington Capitals
a2 Tampa Bay Lightning Toronto Maple Leafs
m2 New York Rangers Pittsburgh Penguins
c1 Colorado Avalanche Dallas Stars
p1 Calgary Flames Nashville Predators
c2 St. Louis Blues Minnesota Wild
p2 Los Angeles Kings Vegas Golden Knights


In [70]:
upd.display()

CGY-0 vs 0-NSH                                                                           CAR-0 vs 0-WSH
               p                                                           m                             
LAK-0 vs 0-VGK                              sc                                           NYR-0 vs 0-PIT
                              sc1                           sc2                                          
COL-0 vs 0-DAL                                                                           FLA-0 vs 0-BOS
               c                                                           a                             
STL-0 vs 0-MIN                                                                           TBL-0 vs 0-TOR


In [ ]:
upd._teams[15]

In [12]:
url = 'https://api.nhle.com/stats/rest/en/franchise?sort=fullName&include=lastSeason.id&include=firstSeason.id'
team_res = requests.get(url).json()

In [13]:
teams = team_res["data"]

In [10]:
requests.get("https://api.nhle.com/stats/rest/en/config").json()


'https://api.nhle.com/stats/rest/en/team/summary?isAggregate=false&isGame=false&sort=%5B%7B%22property%22:%22points%22,%22direction%22:%22DESC%22%7D,%7B%22property%22:%22wins%22,%22direction%22:%22DESC%22%7D,%7B%22property%22:%22teamId%22,%22direction%22:%22ASC%22%7D%5D&start=0&limit=50&factCayenneExp=gamesPlayed%3E=1&cayenneExp=gameTypeId=2%20and%20seasonId%3C=20232024%20and%20seasonId%3E=2023202'

In [10]:
path = "https://api.nhle.com/stats/rest/en/team/summary?isAggregate=false&isGame=false&sort=%5B%7B%22property%22:%22points%22,%22direction%22:%22DESC%22%7D,%7B%22property%22:%22wins%22,%22direction%22:%22DESC%22%7D,%7B%22property%22:%22teamId%22,%22direction%22:%22ASC%22%7D%5D&start=0&limit=50&factCayenneExp=gamesPlayed%3E=1&cayenneExp=gameTypeId=2%20and%20seasonId%3C=20232024%20and%20seasonId%3E=2023202"
teams = requests.get(path).json()

In [17]:
path = "https://api.nhle.com/stats/rest/en/season?sort=%5B%7B%22property%22:%22id%22,%22direction%22:%22DESC%22%7D%5D"
seasons = requests.get(path).json()

In [14]:
path = "https://api-web.nhle.com/v1/standings/now"
standings_data = requests.get(path).json()

In [64]:
standings = {'Eastern': {'Atlantic': [], 'Metropolitan': [], 'teams': []}, 'Western': {'Central': [], 'Pacific': [], 'teams': []}, 'teams': []}
# for team in standings["standings"]:
#     print(team["teamAbbrev"]["default"], team["conferenceAbbrev"], team["conferenceSequence"], team["divisionAbbrev"], team["divisionSequence"], team["leagueSequence"])

league = sorted(list(standings_data["standings"]), key=lambda k: int(k["divisionSequence"]))
for team in league:
    standings['teams'].append(team)
    standings[team['conferenceName']]['teams'].append(team)
    standings[team['conferenceName']][team['divisionName']].append(team)
standings['teams'] = sorted(standings['teams'], key=lambda k: int(k['leagueSequence']))

standings['Eastern']['teams'] = sorted(standings['Eastern']['teams'], key=lambda k: int(k['conferenceSequence']))
standings['Western']['teams'] = sorted(standings['Western']['teams'], key=lambda k: int(k['conferenceSequence']))

standings['Eastern']['Atlantic'] = sorted(standings['Eastern']['Atlantic'], key=lambda k: int(k['divisionSequence']))
standings['Eastern']['Metropolitan'] = sorted(standings['Eastern']['Metropolitan'], key=lambda k: int(k['divisionSequence']))
standings['Western']['Central'] = sorted(standings['Western']['Central'], key=lambda k: int(k['divisionSequence']))
standings['Western']['Pacific'] = sorted(standings['Western']['Pacific'], key=lambda k: int(k['divisionSequence']))

In [26]:
team_name = standings_data["standings"][0]["teamName"]["default"]
team_ids = {}
for team in teams:
    team_ids[team["fullName"]] = team["id"]
team_ids[team_name]

33

In [90]:
teams_obj = []
for team in standings_data["standings"]:
    standings_obj = {"conferenceRank": team["conferenceSequence"], "divisionRank": team["divisionSequence"], "leagueRank": team["leagueSequence"], "wildCardRank": team["wildcardSequence"]}
    conference_obj = {"name": team["conferenceName"]}
    division_obj = {"name": team["divisionName"]}
    info_obj = {"name": team["teamName"]["default"], "conference": conference_obj, "division": division_obj}
    team_obj = {"standings": standings_obj, "info": info_obj }
    teams_obj.append(team_obj)
    print(team["teamAbbrev"]["default"], team["conferenceName"], team["conferenceSequence"], team["divisionName"], team["divisionSequence"], team["leagueSequence"])

FLA Eastern 1 Atlantic 1 1
NYR Eastern 2 Metropolitan 1 2
BOS Eastern 3 Atlantic 2 3
VAN Western 1 Pacific 1 4
DAL Western 2 Central 1 5
WPG Western 3 Central 2 6
COL Western 4 Central 3 7
TOR Eastern 4 Atlantic 3 8
CAR Eastern 5 Metropolitan 2 9
EDM Western 5 Pacific 2 10
VGK Western 6 Pacific 3 11
LAK Western 7 Pacific 4 12
DET Eastern 6 Atlantic 4 13
NSH Western 8 Central 4 14
TBL Eastern 7 Atlantic 5 15
PHI Eastern 8 Metropolitan 3 16
NYI Eastern 9 Metropolitan 4 17
CGY Western 9 Pacific 5 18
STL Western 10 Central 5 19
WSH Eastern 10 Metropolitan 5 20
NJD Eastern 11 Metropolitan 6 21
SEA Western 11 Pacific 6 22
PIT Eastern 12 Metropolitan 7 23
BUF Eastern 13 Atlantic 6 24
MIN Western 12 Central 6 25
MTL Eastern 14 Atlantic 7 26
ARI Western 13 Central 7 27
OTT Eastern 15 Atlantic 8 28
CBJ Eastern 16 Metropolitan 8 29
ANA Western 14 Pacific 7 30
SJS Western 15 Pacific 8 31
CHI Western 16 Central 8 32


In [91]:
standings = {'Eastern': {'Atlantic': [], 'Metropolitan': [], 'teams': []}, 'Western': {'Central': [], 'Pacific': [], 'teams': []}, 'teams': []}

league = sorted(teams_obj, key=lambda k: int(k['standings']['divisionRank']))
for team in league:
    standings['teams'].append(team)
    standings[team['info']['conference']['name']]['teams'].append(team)
    standings[team['info']['conference']['name']][team['info']['division']['name']].append(team)
standings['teams'] = sorted(standings['teams'], key=lambda k: int(k['standings']['leagueRank']))

standings['Eastern']['teams'] = sorted(standings['Eastern']['teams'], key=lambda k: int(k['standings']['conferenceRank']))
standings['Western']['teams'] = sorted(standings['Western']['teams'], key=lambda k: int(k['standings']['conferenceRank']))

standings['Eastern']['Atlantic'] = sorted(standings['Eastern']['Atlantic'], key=lambda k: int(k['standings']['divisionRank']))
standings['Eastern']['Metropolitan'] = sorted(standings['Eastern']['Metropolitan'], key=lambda k: int(k['standings']['divisionRank']))
standings['Western']['Central'] = sorted(standings['Western']['Central'], key=lambda k: int(k['standings']['divisionRank']))
standings['Western']['Pacific'] = sorted(standings['Western']['Pacific'], key=lambda k: int(k['standings']['divisionRank']))

In [92]:
ealeader = standings['Eastern']['Atlantic'][0]
emleader = standings['Eastern']['Metropolitan'][0]
wcleader = standings['Western']['Central'][0]
wpleader = standings['Western']['Pacific'][0]
for team in standings['Eastern']['teams']:
    if int(team['standings']['wildCardRank']) == 1:
        e1wild = team
    if int(team['standings']['wildCardRank']) == 2:
        e2wild = team

for team in standings['Western']['teams']:
    if int(team['standings']['wildCardRank']) == 1:
        w1wild = team
    if int(team['standings']['wildCardRank']) == 2:
        w2wild = team

if int(ealeader['standings']['conferenceRank']) < int(emleader['standings']['conferenceRank']):
    a1_teams = (ealeader, e2wild)
    m1_teams = (emleader, e1wild)
else:
    a1_teams = (ealeader, e1wild)
    m1_teams = (emleader, e2wild)
a2_teams = (standings['Eastern']['Atlantic'][1], standings['Eastern']['Atlantic'][2])
m2_teams = (standings['Eastern']['Metropolitan'][1], standings['Eastern']['Metropolitan'][2])

if int(wcleader['standings']['conferenceRank']) < int(wpleader['standings']['conferenceRank']):
    c1_teams = (wcleader, w2wild)
    p1_teams = (wpleader, w1wild)
else:
    c1_teams = (wcleader, w1wild)
    p1_teams = (wpleader, w2wild)
c2_teams = (standings['Western']['Central'][1], standings['Western']['Central'][2])
p2_teams = (standings['Western']['Pacific'][1], standings['Western']['Pacific'][2])

print(f"a1 {a1_teams[0]['info']['name']} {a1_teams[1]['info']['name']}")
print(f"m1 {m1_teams[0]['info']['name']} {m1_teams[1]['info']['name']}")
print(f"a2 {a2_teams[0]['info']['name']} {a2_teams[1]['info']['name']}")
print(f"m2 {m2_teams[0]['info']['name']} {m2_teams[1]['info']['name']}")

print(f"c1 {c1_teams[0]['info']['name']} {c1_teams[1]['info']['name']}")
print(f"p1 {p1_teams[0]['info']['name']} {p1_teams[1]['info']['name']}")
print(f"c2 {c2_teams[0]['info']['name']} {c2_teams[1]['info']['name']}")
print(f"p2 {p2_teams[0]['info']['name']} {p2_teams[1]['info']['name']}")

a1 Florida Panthers Tampa Bay Lightning
m1 New York Rangers Detroit Red Wings
a2 Boston Bruins Toronto Maple Leafs
m2 Carolina Hurricanes Philadelphia Flyers
c1 Dallas Stars Los Angeles Kings
p1 Vancouver Canucks Nashville Predators
c2 Winnipeg Jets Colorado Avalanche
p2 Edmonton Oilers Vegas Golden Knights


In [93]:
standings_data["standings"][0]

{'conferenceAbbrev': 'E',
 'conferenceHomeSequence': 3,
 'conferenceL10Sequence': 2,
 'conferenceName': 'Eastern',
 'conferenceRoadSequence': 1,
 'conferenceSequence': 1,
 'date': '2024-03-03',
 'divisionAbbrev': 'A',
 'divisionHomeSequence': 2,
 'divisionL10Sequence': 2,
 'divisionName': 'Atlantic',
 'divisionRoadSequence': 1,
 'divisionSequence': 1,
 'gameTypeId': 2,
 'gamesPlayed': 61,
 'goalDifferential': 57,
 'goalDifferentialPctg': 0.934426,
 'goalAgainst': 145,
 'goalFor': 202,
 'goalsForPctg': 3.311475,
 'homeGamesPlayed': 30,
 'homeGoalDifferential': 17,
 'homeGoalsAgainst': 81,
 'homeGoalsFor': 98,
 'homeLosses': 8,
 'homeOtLosses': 2,
 'homePoints': 42,
 'homeRegulationPlusOtWins': 19,
 'homeRegulationWins': 15,
 'homeTies': 0,
 'homeWins': 20,
 'l10GamesPlayed': 10,
 'l10GoalDifferential': 25,
 'l10GoalsAgainst': 14,
 'l10GoalsFor': 39,
 'l10Losses': 1,
 'l10OtLosses': 0,
 'l10Points': 18,
 'l10RegulationPlusOtWins': 8,
 'l10RegulationWins': 6,
 'l10Ties': 0,
 'l10Wins': 9,

In [11]:
teams


   'teamId': 15,
   'ties': None,
   'wins': 55,
   'winsInRegulation': 45,
   'winsInShootout': 2},
  {'faceoffWinPct': None,
   'gamesPlayed': 80,
   'goalsAgainst': 250,
   'goalsAgainstPerGame': 3.125,
   'goalsFor': 385,
   'goalsForPerGame': 4.8125,
   'losses': 16,
   'otLosses': None,
   'penaltyKillNetPct': 0.84756,
   'penaltyKillPct': 0.798781,
   'pointPct': 0.7375,
   'points': 118,
   'powerPlayNetPct': 0.267361,
   'powerPlayPct': 0.277777,
   'regulationAndOtWins': 54,
   'seasonId': 19811982,
   'shotsAgainstPerGame': 30.075,
   'shotsForPerGame': 30.8625,
   'teamFullName': 'New York Islanders',
   'teamId': 2,
   'ties': 10,
   'wins': 54,
   'winsInRegulation': 54,
   'winsInShootout': 0},
  {'faceoffWinPct': 0.513365,
   'gamesPlayed': 82,
   'goalsAgainst': 192,
   'goalsAgainstPerGame': 2.34146,
   'goalsFor': 270,
   'goalsForPerGame': 3.29268,
   'losses': 16,
   'otLosses': 4,
   'penaltyKillNetPct': 0.859649,
   'penaltyKillPct': 0.827486,
   'pointPct': 0.7

In [2]:
path = 'https://api-web.nhle.com/v1/club-schedule-season/nyr/20232024'
team_schedule = requests.get(path).json()

In [ ]:
game = date['games'][0]
game_home_id = game['teams']['home']['team']['id']
game_away_id = game['teams']['away']['team']['id']
if game_home_id == away:
    print(game['gameDate'], game['teams']['away']['score'], game['teams']['home']['score'])
    if int(game['teams']['home']['score']) > int(game['teams']['away']['score']):
        result['away_win'] = result['away_win'] + 1
    elif int(game['teams']['home']['score']) < int(game['teams']['away']['score']):
        result['home_win'] = result['home_win'] + 1
    result['matchs'].append({'home': int(game['teams']['away']['score']), 'away': int(game['teams']['home']['score'])})
if game_away_id == away:
    print(game['gameDate'], game['teams']['home']['score'], game['teams']['away']['score'])
    if int(game['teams']['home']['score']) > int(game['teams']['away']['score']):
        result['home_win'] = result['home_win'] + 1
    elif int(game['teams']['home']['score']) < int(game['teams']['away']['score']):
        result['away_win'] = result['away_win'] + 1
    result['matchs'].append({'home': int(game['teams']['home']['score']), 'away': int(game['teams']['away']['score'])})
else:
print('No date in get matchup season')

In [10]:
games = []
for game_data in team_schedule["games"]:
    if game_data["gameState"] == "OFF":
        home_team = {"id": game_data["homeTeam"]["abbrev"]}
        home_team_game = {"team": home_team, "score": game_data["homeTeam"]["score"]}
        away_team = {"id": game_data["awayTeam"]["abbrev"]}
        away_team_game = {"team": home_team, "score": game_data["awayTeam"]["score"]}
        game = {"teams": {"home": home_team_game, "away": away_team_game}}
        games.append(game)
schedules = {"dates": {"games": games}}

In [3]:
for game_data in team_schedule["games"]:
    if game_data["gameType"] == 3:
        pprint.pprint(game_data)

{'awayTeam': {'abbrev': 'WSH',
              'awaySplitSquad': False,
              'darkLogo': 'https://assets.nhle.com/logos/nhl/svg/WSH_dark.svg',
              'id': 15,
              'logo': 'https://assets.nhle.com/logos/nhl/svg/WSH_light.svg',
              'placeName': {'default': 'Washington'},
              'score': 1},
 'easternUTCOffset': '-04:00',
 'gameCenterLink': '/gamecenter/wsh-vs-nyr/2024/04/21/2023030131',
 'gameDate': '2024-04-21',
 'gameOutcome': {'lastPeriodType': 'REG'},
 'gameScheduleState': 'OK',
 'gameState': 'OFF',
 'gameType': 3,
 'homeTeam': {'abbrev': 'NYR',
              'darkLogo': 'https://assets.nhle.com/logos/nhl/svg/NYR_dark.svg',
              'homeSplitSquad': False,
              'id': 3,
              'logo': 'https://assets.nhle.com/logos/nhl/svg/NYR_light.svg',
              'placeName': {'default': 'Rangers'},
              'score': 4},
 'id': 2023030131,
 'neutralSite': False,
 'periodDescriptor': {'periodType': 'REG'},
 'season': 20232024,
